In [63]:
from math import log
import operator
import copy
import pickle

In [64]:
# 构建一个简单的数据集
def create_data_set():
    data_set = [[1, 1, 'yes'],
                [1, 1, 'yes'],
                [1, 0, 'no'],
                [0, 1, 'no'],
                [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']  # 0,1所对应的特征名
    return data_set, labels
data_set ,labels= create_data_set()

In [65]:
# 计算给定数据集的香浓熵
def calc_shannon_entropy(data_set):
    num_enties = len(data_set)
    label_count={}
    for entry in data_set:
        current_label = entry[-1] # 每一条记录的最后一个元素，为他的label
        label_count[current_label] = label_count.get(current_label,0) +1 # {'yes': 2, 'no': 3}
    shannon_entropy =0.0
    for label in label_count:
        prob = label_count[label] / num_enties
        shannon_entropy += -prob*log(prob,2) 
    return shannon_entropy

In [66]:
"""
data_set为原始数据集,axis为进行划分的维度（特征）,value为该维度的特征值,返回满足该特征值的所有列表的集合
如：原始数据集为
[[1, 1, 'yes'], 
[1, 1, 'yes'], 
[1, 0, 'no'], 
[0, 1, 'no'], 
[0, 1, 'no']]
函数参数值为split_data_set(data, 0, 0) ; 返回:[[1, 'no'], [1, 'no']]
"""
def split_data_set(data_set,axis,value):
    result_set = []
    for entry in data_set:
        if entry[axis] == value:
            new_entry = entry[:axis] # 像extend，append，remove等方法，都是直接在列表上进行修改，返回为None 别用a=a.append(b)
            new_entry.extend(entry[axis+1:])
            result_set.append(new_entry)
    return result_set

In [67]:
"""
choose_best_feat_to_split 函数思想
    1.统计数据集中的特征数，逐个特征计算分裂后的信息增益；
    2.计算信息增益过程如下：
        a.统计特征A的取值(去重)
        b.根据特征值样本分为几类，A=a1的样本为1类，A=a2的样本类为1类。。。
        c.统计划分之后，每一类的样本数与总样本数的比值，再计算每一类的信息熵，最终加权求和
    3.信息增益= 原始信息熵 - 分类后的信息熵，取最大新兴增益的那个特征作为分类特征
"""
def choose_best_feat_to_split(data_set):
    num_of_feature = len(data_set[0])-1 #最后一列为label 
    base_entropy = calc_shannon_entropy(data_set) # 计算传进来的数据集的信息熵，而不是固定的原始数据集信息熵
    best_info_gain =0.0
    best_feature = -1
    for i in range(num_of_feature):
        feat_value_list = [entry[i] for entry in data_set] # 相当于是去除第i列的所有元素值，组成列表 ['yes', 'yes', 'no', 'no', 'no']
        unique_feat_value = set(feat_value_list)
        new_entropy=0.0
        for feat_value in unique_feat_value:
            sub_data_set = split_data_set(data_set,i,feat_value)
            prob = len(sub_data_set)/float(len(data_set))
            new_entropy += prob*calc_shannon_entropy(sub_data_set)
        info_gain = base_entropy - new_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = i
    return best_feature                                  

In [68]:
# 函数功能：统计 class_list中，出现次数最多的类别
def majority_vote(class_list):
    class_count={}
    for element in class_list:
        class_count[element] = class_count.get(element,0) + 1
    sorted_class_count = sorted(class_count.items(),key=operator.itemgetter(1),reverse=True)
    return sorted_class_count[0][0]

In [69]:
# create_tree函数说明：
# 输入示例：data_set = [[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']] ,
#         labels = ['no surfacing', 'flippers']
# 输出示例：{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
# 算法步骤：
"""
    1.提取传入的数据集中的所有类别元素，如果所有元素类别相同，直接返回该类别作为这个分支结束
    2.如果传进来的数据集中，每一行只剩一个元素，即所有特征都划分完了，只剩下标签类，那么用多数投票原则，返回该最多的类别值，作为这个分支的结果
    3.在两步没有返回的话，下面是递归的主要过程：
        a.选择该数据集的最佳分割特征，找到该特征对应的特征名(主要是便于最终生成树)
        b.创建树，字典结果，键名为最佳分割特征，键值为{}(递归到最后是类别名)
        c.删除一分格的特征名，找到最佳分割特征所对应的特征值，将数据集分为几个部分，每个部分都不再宝行此特征字段。
        d.递归的对分割的每一个数据集进行create_tree操作
    4.返回my_tree 字典结构
"""
def create_tree(data_set,labels):
    class_list = [entry[-1] for entry in data_set]
    # 如果class_list中，第一个元素的个数等于总长度，说明该数据集集中，所有元素类别相同，则停止进行分类。
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    # len(data_set(0)) == 1 说明数据集只剩类标签，如[['yes'],['no'],['yes']] 没办法再分割数据集了
    if len(data_set[0]) == 1:
        return majority_vote(class_list)
    best_feat = choose_best_feat_to_split(data_set)
    best_feat_label = labels[best_feat]
    my_tree = {best_feat_label:{}}
    del labels[best_feat] # 删除特征名中的改元素，已经使用过不能再用
    feat_values= [entry[best_feat] for entry in data_set]
    unique_vals=set(feat_values)
    for value in unique_vals:
        sub_features = labels
        my_tree[best_feat_label][value] = create_tree(split_data_set(data_set,best_feat,value),sub_features)
    return my_tree

In [74]:
# classify函数说明：
# 输入示例: input_tree : {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
#          feat_labels : ['no surfacing','flippers']
#          test_vec : [1,1]
# 输出示例: yes
# 算法思想
"""
我们在创建决策树的时候，知道最终的类标签一定是在叶子节点上，用字典结果来说，就是最底层的value值，而就算是最简单的决策树。如：
{'happy':{0:'no',1:'yes'}} 也要获取嵌套在里面的字典。
所以classify实际上也就是一个递归函数，一直读到不再是dict结果的key为止

步骤：
    1.获取最外层的key，然后提取所对应的字典结构
    2.找到上一步的key(特征名)所对应的索引值，便于我们在test_vec中找到对应的特征值
    3.遍历第一步或得到的字典结构的keys，如果key对应的value仍是字典结构，则递归调用classify函数：
"""
def classify(input_tree, feat_labels, test_vec):
    first_key = list(input_tree.keys())[0]  # 字典的key不支持索引，所以转成list形式
    second_dict = input_tree[first_key]
    feat_index = feat_labels.index(first_key)
    for key in second_dict.keys():
        if test_vec[feat_index] == key:
            if type(second_dict[key]).__name__ == 'dict':
                class_label = classify(second_dict[key], feat_labels, test_vec)
            else:
                class_label = second_dict[key]
    return class_label

In [75]:
# 存储决策树到硬盘
def store_tree(input_tree, file_name):
    # 这里的mode,要写作'wb'
    fw = open(file_name, 'wb')
    pickle.dump(input_tree, fw)
    fw.close()

In [76]:
# 从硬盘中加载决策树
def load_tree(file_name):
    fr = open(file_name, 'rb')
    return pickle.load(fr)

In [77]:
data,feat_labels = create_data_set()
# 小知识：python 中，"a = b"表示的是对象 a 引用对象 b，对象 a 本身没有单独分配内存空间(重要：不是复制！)
# 这里对feat_labels进行拷贝操作，因为在create_tree的算法中，对feat_labels进行删除操作   
backup = copy.copy(feat_labels)
my_tree = create_tree(data, feat_labels)
print(my_tree)
store_tree(my_tree,'my_decision_tree.txt')
result = classify(my_tree,backup,[1,1])
if result == 'yes':
    print("该物种是鱼类")
else:
    print('该物种不是鱼类')
    

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
该物种是鱼类
